##### Read the JSON file using the spark dataframe reader

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
constructors_schema = StructType(fields=[StructField("constructorId",IntegerType(),False),
                                         StructField("constructorRef",StringType(),True),
                                         StructField("name",StringType(),True),
                                         StructField("nationality",StringType(),True),
                                         StructField("url",StringType(),True)
                                         ])

In [0]:
constructor_df = spark.read \
.schema(constructors_schema) \
.json(f"{raw_folder_path}/{v_file_date}/constructors.json")

##### Drop unwanted columns from the dataframe

In [0]:
from pyspark.sql.functions import col

In [0]:
constructor_dropped_df = constructor_df.drop(col('url'))

##### Rename columns and add ingestion date

In [0]:
from pyspark.sql.functions import lit

In [0]:
constructor_renamed_df = constructor_dropped_df.withColumnRenamed("constructorId", "constructor_id") \
                                             .withColumnRenamed("constructorRef", "constructor_ref") \
                                             .withColumn("file_date", lit(v_file_date))

In [0]:
constructor_final_df = add_ingestion_date(constructor_renamed_df)

##### Write output to container in delta format

In [0]:
constructor_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.constructors")